In [1]:
import networkx as nx
import ndlib.models.epidemics as ep
import ndlib.models.ModelConfig as mc
import ndlib.models.CompositeModel as gc
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend
from ndlib.viz.mpl.DiffusionPrevalence import DiffusionPrevalence
from pymongo import MongoClient
import numpy as np
from ndlibCustom.SEIR_ASModel import SEIR_ASModel
from statsmodels.distributions.empirical_distribution import ECDF
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import seaborn as sns
import scipy.stats as st
from IPython.display import clear_output

no display found. Using non-interactive Agg backend


In [2]:
import src.dataset_generators.sezioni_generator
import src.dataset_generators.schools_generator
import src.layer_generators.household_layer
import src.layer_generators.school_layer
import src.layer_generators.work_layer
import src.graph_generators.graph_generator_full

client = MongoClient('localhost', 27017)

In [3]:
def getWorkersDist():
    client = MongoClient('localhost', 27017)
    db = client['tesi']
    db_sezioni = db['sezioni']
    db_lavoro = db['lavoro']

    work_dist = {}
    #work_dist = []
    for i in list(db_sezioni.find()):
        #print("Indexing sez:",i["properties"]["SEZ"], end="\r", flush=True)
        num_attivi = 0
        num_workers = 0
        if "famiglie" in i.keys():
            for fam in i["famiglie"]:
                for mem in fam["members"]:
                    if mem["age"] == "20-64": 
                        num_attivi += 1
                        if "sez_lavoro" in mem.keys(): num_workers += 1
            if (num_workers != 0 and num_attivi != 0):
                work_dist[i["properties"]["SEZ"]] = (num_workers/num_attivi)*100
                #work_dist.append({"sez":i["properties"]["SEZ"],"num":(num_workers/num_attivi)*100})

    return work_dist
    #list_dist =  [i["num"] for i in work_dist]
    #print("Mean percentual of workers for sez:",np.mean(list_dist))
    #return list_dist

def getAgeDistrib(defgraph,gc):
    agedistrib_g1 = []
    a1 = 0
    a2 = 0
    a3 = 0
    for i in defgraph.nodes():
        if defgraph.nodes[i]['age'] == '65+':
            a3 += 1
        elif defgraph.nodes[i]['age'] == '20-64':
            a2 += 1
        else: 
            a1 +=1
    print("Distribuzione di età su grafo totale")
    print("Giovani:",a1,"Attivi:",a2,"Anziani:",a3)
    print()
    g1 = a1
    g2 = a2
    g3 = a3
    for i in gc.nodes():
        if gc.nodes[i]['age'] == '65+':
            a3 -= 1
        elif gc.nodes[i]['age'] == '20-64':
            a2 -= 1
        else: 
            a1 -=1
    return [a1,a2,a3,g1,g2,g3]
    #print("Disribuzione di età fuori dalla Giant Component (in percentuale sul totale)")
    #print("Giovani:",a1,"(",round((a1/g1)*100,3),")","Attivi:",a2,"(",round((a2/g2)*100,3),")","Anziani:",a3,"(",round((a3/g3)*100,3),")")
    #print()
    #print("Nodi appartenenti alla Giant Component")
    #print("Giovani:",g1-a1,"Attivi:",g2-a2,"Anziani:",g3-a3)
    
def getLastIterationStatusesByAge(simdata, defgraph):
    iteration = simdata
    graph_stat = {}
    
    for x in iteration[0]['status']:
        graph_stat[x] = iteration[0]['status'][x]
    for iter in iteration:
        for s in iter['status']:
            graph_stat[s] = iter['status'][s]    

    stats_giovani = {0:[],1:[],2:[],3:[],4:[],5:[]}
    stats_attivi = {0:[],1:[],2:[],3:[],4:[],5:[]}
    stats_anziani = {0:[],1:[],2:[],3:[],4:[],5:[]}
    
    for i in graph_stat:
        if defgraph.nodes[i]['age'] == '65+':
            stats_anziani[graph_stat[i]].append(i)
        elif defgraph.nodes[i]['age'] == '20-64':
            stats_attivi[graph_stat[i]].append(i)
        else:
            stats_giovani[graph_stat[i]].append(i)
    
    return {'giovani':stats_giovani,'attivi':stats_attivi,'anziani':stats_anziani}

def getNeighborhoodGC(simdata, GC, defgraph):
    iteration = simdata
    nodelist = []
    neighbors = {}
    graph_stat = {}
    neighbors_s = {}
    neighbors_r = {}
                     
    #Rebuild current iteration status tree
    for x in iteration[0]['status']:
        graph_stat[x] = iteration[0]['status'][x]
    for iter in iteration:
        for s in iter['status']:
            graph_stat[s] = iter['status'][s]   
   
                     #Cycle through nodes
    for i in GC.nodes():
        if i in graph_stat.keys():
            #Process selected S_Nodes
            if graph_stat[i] == 0:
                nodelist.append(i)
                neighborhood = [n for n in defgraph.neighbors(i)]
                neighbors_s[i] = []
                neighbors_r[i] = []
                sum = 0
                sus = 0
                rem = 0
                for n in neighborhood:
                    neighbors[n] = graph_stat[n]
                    sum += 1
                    if graph_stat[n] == 0: sus += 1
                    if graph_stat[n] == 5: rem += 1
                neighbors_s[i].append(int((sus/sum)*100))
                neighbors_r[i].append(int((rem/sum)*100))
    return {'nodelist':nodelist,'neighbors':neighbors,'s':neighbors_s,'r':neighbors_r}

def buildSimulationProgressionByAge(simdata, defgraph):
    graph_stat = {}
    giovani_a = {}
    giovani_s = {}
    attivi_a = {}
    attivi_s = {}
    anziani_a = {}
    anziani_s = {}

    #Build graph_stat with starting statuses
    for i in simdata[0]['status']:
        graph_stat[i] = simdata[0]['status'][i]

    #Rebuild simulations by age and status evolution
    for iter in simdata:
        inc_1a = 0
        inc_1s = 0
        inc_2a = 0
        inc_2s = 0
        inc_3a = 0
        inc_3s = 0
        for s in iter['status']:
            if graph_stat[s] == 0 or graph_stat[s] == 1 or graph_stat[s] == 2:
                if iter['status'][s] == 3:
                    if defgraph.nodes[s]['age'] == '20-64':
                        inc_2a += 1
                    elif defgraph.nodes[s]['age'] == '65+':
                        inc_3a += 1
                    else: 
                        inc_1a += 1
                elif iter['status'][s] == 4:
                    if defgraph.nodes[s]['age'] == '20-64':
                        inc_2s += 1
                    elif defgraph.nodes[s]['age'] == '65+':
                        inc_3s += 1
                    else: 
                        inc_1s += 1
        #Append data of current n-iteration
        giovani_a[iter['iteration']] = inc_1a
        giovani_s[iter['iteration']] = inc_1s
        attivi_a[iter['iteration']] = inc_2a
        attivi_s[iter['iteration']] = inc_2s
        anziani_a[iter['iteration']] = inc_3a
        anziani_s[iter['iteration']] = inc_3s
    return {'giovani_a':giovani_a,'giovani_s':giovani_s,'attivi_a':attivi_a,'attivi_s':attivi_s,'anziani_a':anziani_a,'anziani_s':anziani_s}

In [ ]:
%%capture cap
#CUSTOM SEIR_AS MODEL SIMULATION
print("Simulating...")
#Run 50 simulations, results will be a mean of all data generated
simulations = []

it = 0
while it < 50:
    print("Running simulation number",it+1)
    #Create dist containing all simulation information
    sim = {}
    tot_agedistrib = {"giovani":[],"attivi":[],"anziani":[]}
    G0_agedistrib = {"giovani":[],"attivi":[],"anziani":[]}

    #Clear last iteration's graph DB
    try:
        client.drop_database('tesi')
        print("Database cleared.")
    except:
        print("There is no database currently saved.")

    #Generate new multilayer graph
    src.dataset_generators.sezioni_generator.main()
    src.dataset_generators.schools_generator.main()
    try:
        families_distance = src.layer_generators.household_layer.main()
    except:
        print("Household layer failed to generate. Restarting iteration...")
        continue
    src.layer_generators.school_layer.main()
    src.layer_generators.work_layer.main()
    graph = src.graph_generators.graph_generator_full.main(schools=False) 
        
    # Model selection
    model = SEIR_ASModel(graph)

    # Model Configuration
    cfg = mc.Configuration()
    cfg.add_model_parameter('beta', 0.57) #Infection rate from I_A/S Neighbors
    cfg.add_model_parameter('gamma', 0.15) #Recovery/Removal rate
    cfg.add_model_parameter('alpha', 0.2) #Latent period
    cfg.add_model_parameter('kappa', 0.25) #Symptomatic/Asymptomatic Ratio

    #This parameter "smoothens" out the infection rate in the first iterations.
    #1: 1-(1-beta)^(num_of_infected_neighbors)[0.99 on > 5 infected neighbors]
    #0: beta*(average_infected_weight)
    cfg.add_model_parameter('tp_rate', 0)

    cfg.add_model_parameter("fraction_infected", 0.005) #Starting infected nodes
    model.set_initial_status(cfg)

    lastsim = model.iteration_bunch(60,progress_bar=True)
    for i in lastsim:
        i.pop('node_count')
        i.pop('status_delta')
        
    #Save graph related files
    sim['simulation'] = lastsim
    sim['workers_dist'] = getWorkersDist()
    sim['families_distance'] = families_distance
    sim['density'] = nx.density(graph)
    sim['degree'] = graph.degree()
    sim['last_iteration_status'] = getLastIterationStatusesByAge(lastsim,graph)
    
    Gcc = sorted(nx.connected_components(graph), key=len, reverse=True)
    sim['herd_immunity_gc'] = getNeighborhoodGC(lastsim, graph.subgraph(Gcc[0]), graph)
    
    temp = getAgeDistrib(graph,graph.subgraph(Gcc[0]))
    tot_agedistrib['giovani'].append(temp[0])
    tot_agedistrib['attivi'].append(temp[1])
    tot_agedistrib['anziani'].append(temp[2])
    G0_agedistrib['giovani'].append(temp[3])
    G0_agedistrib['attivi'].append(temp[4])
    G0_agedistrib['anziani'].append(temp[5])
    sim['tot_agedistrib'] = tot_agedistrib
    sim['G0_agedistrib'] = G0_agedistrib
    sim['progression'] = buildSimulationProgressionByAge(lastsim,graph)
    
    simulations.append(sim)
    del(graph) #Remove graph and reclaim memory for next iteration
    #clear_output() #Clear output to avoid cluttering: simulation number will be printed at each iteration
    it += 1

Simulating...
Running simulation number 1
Database cleared.
Generating sezioni...
Sezioni generated. Dumping to db...
Dumping to db done.
Indexing schools...
Indexing schools done. Appending closest schools to sezioni...
Dumping to db done.
TBR has been triggered. Failed to generate sez n 2350
TBR has been triggered. Failed to generate sez n 5072
Sezioni saltate per via di malformazioni tra famiglie e fasce: 59
Sezioni simulate:  1095
1.4200799085919866
Indexing schools...
Done. Calculating mean school size...
Scolari all'infanzia 176
Scolari alle elementari 242
Scolari alle medie 340
Scolari alle superiori 271
Done. Allocating school to under-19...
Current section: 6107 495 53218794391 1374 1392 13961463 1766 1835 1842 1868 2245 2586 2599 2642 3383 3385 34083418 3435 3467 45234604 47034888 5595 57055734 5778 5810 5811 5813 5814 5818 5821 5822 5824 5826 5828 5832 5851 5948 5951 5957 6048 6071 6079 6082 6083 6084 6092
Kids allocated to schools. Generating classes...
Dumping to db done.


100%|██████████| 60/60 [11:35<00:00, 11.60s/it]


Distribuzione di età su grafo totale
Giovani: 204174 Attivi: 720390 Anziani: 305479

Running simulation number 2
Database cleared.
Generating sezioni...
Sezioni generated. Dumping to db...
Dumping to db done.
Indexing schools...
Indexing schools done. Appending closest schools to sezioni...
Dumping to db done.
TBR has been triggered. Failed to generate sez n 2350
TBR has been triggered. Failed to generate sez n 3988
TBR has been triggered. Failed to generate sez n 4291
TBR has been triggered. Failed to generate sez n 5072
Sezioni saltate per via di malformazioni tra famiglie e fasce: 61
Sezioni simulate:  1095
1.4054566916565336
Indexing schools...
Done. Calculating mean school size...
Scolari all'infanzia 177
Scolari alle elementari 241
Scolari alle medie 338
Scolari alle superiori 269
Done. Allocating school to under-19...
Current section: 6107 167273 1195 1381 1818 1855 1906 1907 1935 1938 2236 3423 3665 4014 4015 4137 4296 4350 4390 4394 4395 4396 4399 4663 4674 4676 4678 4680 5075

  2%|▏         | 1/60 [00:02<02:14,  2.28s/it]

In [5]:
#cap.show()
with open('output.txt', 'w') as f:
    f.write(cap.stdout)
print(len(simulations))
print(simulations[0].keys())

5


In [7]:
mean_degree = {}
for i in range(0,max(len(sim['degree']) for sim in simulations)):
    mean_degree[i] = []
    
for sim in simulations:
    for node in dict(sim['degree']).keys():
        mean_degree[node].append(sim['degree'][node])

for i in mean_degree.keys():
    mean_degree[i] = np.mean(mean_degree[i])
    
#Print mean density
print("Mean density")
print(np.mean([sim['density'] for sim in simulations]))
print("CI 95%:",st.t.interval(alpha=0.95, df=len([sim['density'] for sim in simulations])-1, loc=np.mean([sim['density'] for sim in simulations]), scale=st.sem([sim['density'] for sim in simulations])))
print()

#Print mean degree
print("Mean degree")
print(np.mean(list(mean_degree.values())))
print("CI 95%:",st.t.interval(alpha=0.95, df=len(list(mean_degree.values()))-1, loc=np.mean(list(mean_degree.values())), scale=st.sem(list(mean_degree.values()))))
print()

#ECCDF for degree data in loglog scale
cdf = ECDF(list(mean_degree.values()))
x = np.unique(list(mean_degree.values()))
y = cdf(x)
fig_cdf = plt.figure(figsize=(16,8))
axes = fig_cdf.gca()
axes.loglog(x,1-y,marker='o',ms=8, linestyle='--')
axes.set_xlabel('Degree',size=20)
axes.set_ylabel('ECCDF', size=20)
plt.show()
plt.savefig('export_multi_full/noschools_eccdf_degree.png')

Mean density
1.195248527486388e-05
CI 95%: (1.1929369101465121e-05, 1.197560144826264e-05)

Mean degree
14.701510464849722
CI 95%: (14.672608471208152, 14.730412458491292)



In [8]:
mean_workers_dist = {}

for sim in simulations:
    for node in sim['workers_dist'].keys():
        mean_workers_dist[node] = []

for sim in simulations:
    for node in sim['workers_dist'].keys():
        mean_workers_dist[node].append(sim['workers_dist'][node])

for i in mean_workers_dist:
    mean_workers_dist[i] = np.mean(mean_workers_dist[i]) 

print("Mean percentual of workers for sez")
print(np.mean(list(mean_workers_dist.values())))
print("CI 95%:",st.t.interval(alpha=0.95, df=len(list(mean_workers_dist.values()))-1, loc=np.mean(list(mean_workers_dist.values())), scale=st.sem(list(mean_workers_dist.values()))))
print()

#ECDF of workers distribution per sez in linear scale
plt.style.use('default')
cdf = ECDF(list(mean_workers_dist.values()))
x = np.unique(list(mean_workers_dist.values()))
y = cdf(x)
fig_cdf = plt.figure(figsize=(15,10))
axes = fig_cdf.gca()
axes.plot(x,y,marker='o',ms=8,  linestyle='--')
# general title
plt.suptitle('Percentuale di lavoratori sul totale della popolazione 20-64 (per sezione)', fontsize=13, fontweight=0, color='black', style='italic', y=0.92)
axes.set_xlabel('Percentuale',size=20)
axes.set_ylabel('ECDF', size = 20)
plt.show()
plt.savefig('export_multi_full/noschools_ecdf_workers.png')

Mean percentual of workers for sez
49.57929683606824
CI 95%: (49.33552249230883, 49.82307117982764)



In [9]:
#Print mean families distance
families_distance = [np.mean(list(sim['families_distance'].values())) for sim in simulations]
print("Mean families distance for simulated sez")
print(np.mean(families_distance))
print("CI 95%:",st.t.interval(alpha=0.95, df=len(families_distance)-1, loc=np.mean(families_distance), scale=st.sem(families_distance)))
print()

families_distance = {}

for sim in simulations:
    for key in sim['families_distance']:
        families_distance[key] = []

for sim in simulations:
    for key in sim['families_distance']:
        families_distance[key].append(sim['families_distance'][key])

for key in families_distance:
    families_distance[key] = np.mean(families_distance[key])

#np.mean(list(families_distance.values()))
#print(np.mean([np.mean(list(sim['families_distance'].values())) for sim in simulations]))

# ECDF linear scale
plt.style.use('default')
cdf = ECDF(list(families_distance.values()))
x = np.unique(list(families_distance.values()))
y = cdf(x)
fig_cdf = plt.figure(figsize=(15,10))
axes = fig_cdf.gca()
axes.plot(x,y,marker='o',ms=8,  linestyle='--')
#general title
plt.suptitle('Differenza tra sezioni simulate e dataset reale', fontsize=13, fontweight=0, color='black', style='italic', y=0.92)

axes.set_xlabel('Percentuale',size=20)
axes.set_ylabel('ECDF', size = 20)
plt.show()
plt.savefig('export_multi_full/noschools_ecdf_families_distance.png')

Mean families distance for simulated sez
1.4099058098508475
CI 95%: (1.401777704435479, 1.4180339152662158)



In [38]:
#Create complete status map by dynamically rebuilding infection iteration results
#Keys are nodes, values are final statuses
#Look for keys in graph to get attributes 

##START FOR EACH                
#Calculate how many survivors in absolute at last iter
s_giovani = []
r_giovani = []
s_attivi = []
r_attivi = []
s_anziani = []
r_anziani = []

for sim in simulations:
    #Build sum of age groups 
    giovani_p = []
    attivi_p = []
    anziani_p = []
    for i in sim['last_iteration_status']['giovani']:
        giovani_p.append(len(sim['last_iteration_status']['giovani'][i]))
        attivi_p.append(len(sim['last_iteration_status']['attivi'][i]))
        anziani_p.append(len(sim['last_iteration_status']['anziani'][i]))

    #Build mean for CI(95%)
    #List "survivors" at last iteration for all simulations
    s_giovani.append(giovani_p[0])
    r_giovani.append(giovani_p[5])
    s_attivi.append(attivi_p[0])
    r_attivi.append(attivi_p[5])
    s_anziani.append(anziani_p[0])
    r_anziani.append(anziani_p[5])
##END FOR EACH

giovani_p = [np.mean(s_giovani),np.mean(r_giovani)]
attivi_p = [np.mean(s_attivi),np.mean(r_attivi)]
anziani_p = [np.mean(s_anziani),np.mean(r_anziani)]

#TODO: PLOT STATUS AT LAST ITERATION 
plt.clf()
plt.rcParams['figure.figsize'] = [15, 10]
# set width of bar
barWidth = 0.25

# Set position of bar on X axis
r1 = np.arange(len(giovani_p))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]

# Make the plot
plt.bar(r1, giovani_p, color='#7f6d5f', width=barWidth, edgecolor='white', label='0-19')
plt.bar(r2, attivi_p, color='#557f2d', width=barWidth, edgecolor='white', label='20-64')
plt.bar(r3, anziani_p, color='#2d7f5e', width=barWidth, edgecolor='white', label='65+')

# Add xticks on the middle of the group bars
# general title
plt.suptitle("Stato complessivo degli agenti all'ultima iterazione per fasce di età", fontsize=13, fontweight=0, color='black', style='italic', y=0.92)
plt.xlabel('Stato', fontweight='bold')
labels = ['S',"R"]
plt.xticks([r + barWidth for r in range(len(giovani_p))], labels)

# Create legend & Show graphic
plt.legend()
plt.show()
plt.savefig('export_multi_full/noschools_last_iteration_statuses.png')

print()
print("Mean values")
print("Giovani A/S", giovani_p)
print("Attivi A/S", attivi_p)
print("Anziani A/S", anziani_p)
print()
print("CI 95%")
print("Giovani (S):",st.t.interval(alpha=0.95, df=len(s_giovani)-1, loc=np.mean(s_giovani), scale=st.sem(s_giovani)))
print("Giovani (R):",st.t.interval(alpha=0.95, df=len(r_giovani)-1, loc=np.mean(r_giovani), scale=st.sem(r_giovani)))
print("Attivi (S):",st.t.interval(alpha=0.95, df=len(s_attivi)-1, loc=np.mean(s_attivi), scale=st.sem(s_attivi)))
print("Attivi (R):",st.t.interval(alpha=0.95, df=len(r_attivi)-1, loc=np.mean(r_attivi), scale=st.sem(r_attivi)))
print("Anziani (S):",st.t.interval(alpha=0.95, df=len(s_anziani)-1, loc=np.mean(s_anziani), scale=st.sem(s_anziani)))
print("Anziani (R):",st.t.interval(alpha=0.95, df=len(r_anziani)-1, loc=np.mean(r_anziani), scale=st.sem(r_anziani)))


Mean values
Giovani A/S [12143.8, 189473.4]
Attivi A/S [158861.0, 554849.8]
Anziani A/S [290156.4, 15200.4]

CI 95%
Giovani (S): (11983.838686757303, 12303.761313242696)
Giovani (R): (189231.640972715, 189715.159027285)
Attivi (S): (158264.42405728297, 159457.57594271703)
Attivi (R): (554280.1945746725, 555419.4054253276)
Anziani (S): (289903.83412752836, 290408.9658724717)
Anziani (R): (15014.812556456178, 15385.987443543821)


In [11]:
#Check if herd immunity has ben achieved inside the giant component G0

##START FOR EACH
#Collect this data per simulation basis
G0_susceptible = []
G0_neighbors = []
G0_neighbors_s = []
G0_neighbors_r = []
df = pd.DataFrame(columns=['Percentuale', 'Numero di individui', 'S/R'])

for sim in simulations:
    #Create list of sums
    G0_susceptible.append(len(sim['herd_immunity_gc']['nodelist']))
    G0_neighbors.append(sim['herd_immunity_gc']['neighbors'])
    G0_neighbors_s.append(sim['herd_immunity_gc']['s'])
    G0_neighbors_r.append(sim['herd_immunity_gc']['r'])
##END FOR EACH

#Create a list with values to be plotted
dist_temp_s = {'0-10':[],'11-20':[],'21-30':[],'31-40':[],'41-50':[],'51-60':[],'61-70':[],'71-80':[],'81-90':[],'91-100':[]}
dist_temp_r = {'0-10':[],'11-20':[],'21-30':[],'31-40':[],'41-50':[],'51-60':[],'61-70':[],'71-80':[],'81-90':[],'91-100':[]}

#{node:percent}
for i in dist_temp_r:
    interval = i.split('-')
    interval = [int(interval[0]),int(interval[1])]
    #taking interval as [x,x]
    for sim in simulations:
        val_s = 0
        val_r = 0
        for val in list(sim['herd_immunity_gc']['r'].values()):
            if val[0] >= interval[0] and val[0] <= interval[1]:
                val_r += 1
        for val in list(sim['herd_immunity_gc']['s'].values()):
            if val[0] >= interval[0] and val[0] <= interval[1]:
                val_s += 1
        dist_temp_s[i].append(val_s)
        dist_temp_r[i].append(val_r)
        df = df.append({'Percentuale':i,'Numero di individui':val_s,'S/R':'S'},ignore_index=True)
        df = df.append({'Percentuale':i,'Numero di individui':val_r,'S/R':'R'},ignore_index=True)

print(df)
print()
plt.clf()
sns.set_theme(style="whitegrid")
ax = sns.barplot(x="Percentuale", y="Numero di individui", hue="S/R", data=df, ci=95)
plt.savefig('export_multi_full/noschools_G0_herd_immunity.png')

print("Nodi in stato S all'ultima iterazione:",np.mean(G0_susceptible))
print("CI 95%",st.t.interval(alpha=0.95, df=len(G0_susceptible)-1, loc=np.mean(G0_susceptible), scale=st.sem(G0_susceptible)))
print()

print("Numero di neighbors dei nodi con stato S all'ultima iterazione:",np.mean([len(n) for n in G0_neighbors]))
print("CI 95%",st.t.interval(alpha=0.95, df=len([len(n) for n in G0_neighbors])-1, loc=np.mean([len(n) for n in G0_neighbors]), scale=st.sem([len(n) for n in G0_neighbors])))
print()

G0_NS = []
G0_NIE = []
G0_NR = []
for iteration_neighbors in G0_neighbors:
    NS = 0
    NIE = 0
    NR = 0
    for i in iteration_neighbors:
        if iteration_neighbors[i] == 0: NS += 1
        elif iteration_neighbors[i] == 5: NR += 1
        else: NIE +=1
    G0_NS.append(NS)
    G0_NIE.append(NIE)
    G0_NR.append(NR)

print("S:",np.mean(G0_NS),"R:",np.mean(G0_NR),"EAS/IAS:",np.mean(G0_NIE))
print("CI 95%")
print("S",st.t.interval(alpha=0.95, df=len(G0_NS)-1, loc=np.mean(G0_NS), scale=st.sem(G0_NS)))
print("R",st.t.interval(alpha=0.95, df=len(G0_NR)-1, loc=np.mean(G0_NR), scale=st.sem(G0_NR)))
print("EAS/IAS",st.t.interval(alpha=0.95, df=len(G0_NIE)-1, loc=np.mean(G0_NIE), scale=st.sem(G0_NIE)))

   Percentuale Numero di individui S/R
0         0-10                8115   S
1         0-10                9162   R
2         0-10                8253   S
3         0-10                8658   R
4         0-10                8274   S
..         ...                 ...  ..
95      91-100                8005   R
96      91-100                8039   S
97      91-100                7879   R
98      91-100                9059   S
99      91-100                8015   R

[100 rows x 3 columns]

Nodi in stato S all'ultima iterazione: 24335.4
CI 95% (23649.904418628466, 25020.895581371537)

Numero di neighbors dei nodi con stato S all'ultima iterazione: 32317.6
CI 95% (31620.53332594712, 33014.666674052874)

S: 16124.4 R: 15583.0 EAS/IAS: 610.2
CI 95%
S (15475.30264252247, 16773.49735747753)
R (15433.535421489009, 15732.464578510991)
EAS/IAS (553.3852218031055, 667.0147781968946)


In [ ]:
# ECDF linear scale
'''
plt.style.use('default')
cdf = ECDF(neighbors_s)
x = np.unique(neighbors_s)
y = cdf(x)
fig_cdf = plt.figure(figsize=(15,10))
axes = fig_cdf.gca()
axes.plot(x,y,marker='o',ms=6, linestyle='--', label='Percentuale di S')
cdf = ECDF(neighbors_r)
x = np.unique(neighbors_r)
y = cdf(x)
axes.plot(x,y,marker='o',ms=6, linestyle='--', label='Percentuale di R')
# general title
plt.suptitle('Composizione del neighbourhood dei nodi con stato S all ultima iterazione (in percentuale)', fontsize=13, fontweight=0, color='black', style='italic', y=0.92)
axes.set_xlabel('Percentuale',size=20)
axes.set_ylabel('ECDF', size = 20)
axes.legend()
plt.show()
'''

In [ ]:
'''
df=pd.DataFrame({'S':neighbors_s,'R':neighbors_r})

#Style
palette = plt.get_cmap('Set1')
plt.style.use('seaborn-darkgrid')

#Hist only
plt.figure(figsize=(15,10))
s = sns.histplot(df["R"], kde=False, color=palette(0), label="Susceptible")
r = sns.histplot(df["S"], kde=False, color=palette(1), label="Removed")
s.set(xlabel='Percentuale', ylabel='Individui')
plt.legend()
plt.show()

plt.figure(figsize=(15,10))
plt.hist([df["S"], df["R"]], color=[palette(0),palette(1)], label=["Susceptible","Removed"])
plt.legend()
plt.xlabel("Percentuale")
plt.ylabel("Individui")
plt.show()


f, axes = plt.subplots(1, 2, figsize=(15, 10), sharex=True)
s = sns.histplot(df["R"], ax=axes[0], color=palette(0))
r = sns.histplot(df["S"], ax=axes[1], color=palette(1))
s.set(xlabel='Percentuale', ylabel='Individui')
r.set(xlabel='Percentual', ylabel='Individui')
plt.show()
'''

In [33]:
sim_giovani_a = []
sim_giovani_s = []
sim_attivi_a = []
sim_attivi_s = []
sim_anziani_a = []
sim_anziani_s = []

#Store values of each simulation
for sim in simulations:
    sim_giovani_a.append(sim['progression']['giovani_a'])
    sim_giovani_s.append(sim['progression']['giovani_s'])
    sim_attivi_a.append(sim['progression']['attivi_a'])
    sim_attivi_s.append(sim['progression']['attivi_s'])
    sim_anziani_a.append(sim['progression']['anziani_a'])
    sim_anziani_s.append(sim['progression']['anziani_s'])

#Calculate mean values of each class
CI_giovani_a = {}
CI_giovani_s = {}
CI_attivi_a = {}
CI_attivi_s = {}
CI_anziani_a = {}
CI_anziani_s = {}

giovani_a = {}
giovani_s = {}
attivi_a = {}
attivi_s = {}
anziani_a = {}
anziani_s = {}

for i in range(0,len(sim_giovani_a[0])):#Num of iterations per simulation (60)
    values = []
    for index in range (0,len(sim_giovani_a)):#Num of simulations (50)
        values.append(sim_giovani_a[index][i])
    giovani_a[i] = np.mean(values)
    CI_giovani_a[i] = st.t.interval(alpha=0.95, df=len(values)-1, loc=np.mean(values), scale=st.sem(values))
    if np.isnan(CI_giovani_a[i][0]) == True and np.isnan(CI_giovani_a[i][1]) == True: CI_giovani_a[i] = (0,0)
    
for i in range(0,len(sim_giovani_s[0])):
    values = []
    for index in range (0,len(sim_giovani_s)):
        values.append(sim_giovani_s[index][i])
    giovani_s[i] = np.mean(values)
    CI_giovani_s[i] = st.t.interval(alpha=0.95, df=len(values)-1, loc=np.mean(values), scale=st.sem(values))
    if np.isnan(CI_giovani_s[i][0]) == True and np.isnan(CI_giovani_s[i][1]) == True: CI_giovani_s[i] = (0,0)
        
for i in range(0,len(sim_attivi_a[0])):
    values = []
    for index in range (0,len(sim_attivi_a)):
        values.append(sim_attivi_a[index][i])
    attivi_a[i] = np.mean(values)
    CI_attivi_a[i] = st.t.interval(alpha=0.95, df=len(values)-1, loc=np.mean(values), scale=st.sem(values))
    if np.isnan(CI_attivi_a[i][0]) == True and np.isnan(CI_attivi_a[i][1]) == True: CI_attivi_a[i] = (0,0)

for i in range(0,len(sim_attivi_s[0])):
    values = []
    for index in range (0,len(sim_attivi_s)):
        values.append(sim_attivi_s[index][i])
    attivi_s[i] = np.mean(values)
    CI_attivi_s[i] = st.t.interval(alpha=0.95, df=len(values)-1, loc=np.mean(values), scale=st.sem(values))
    if np.isnan(CI_attivi_s[i][0]) == True and np.isnan(CI_attivi_s[i][1]) == True: CI_attivi_s[i] = (0,0)

for i in range(0,len(sim_anziani_a[0])):
    values = []
    for index in range (0,len(sim_anziani_a)):
        values.append(sim_anziani_a[index][i])
    anziani_a[i] = np.mean(values)
    CI_anziani_a[i] = st.t.interval(alpha=0.95, df=len(values)-1, loc=np.mean(values), scale=st.sem(values))
    if np.isnan(CI_anziani_a[i][0]) == True and np.isnan(CI_anziani_a[i][1]) == True: CI_anziani_a[i] = (0,0)

for i in range(0,len(sim_anziani_s[0])):
    values = []
    for index in range (0,len(sim_anziani_s)):
        values.append(sim_anziani_s[index][i])
    anziani_s[i] = np.mean(values)
    CI_anziani_s[i] = st.t.interval(alpha=0.95, df=len(values)-1, loc=np.mean(values), scale=st.sem(values))
    if np.isnan(CI_anziani_s[i][0]) == True and np.isnan(CI_anziani_s[i][1]) == True: CI_anziani_s[i] = (0,0)
        
# Make a data frame
dataframes = []
dataframes.append(pd.DataFrame({'x': range(1,len(simulations[0]['simulation'])+1),'Giovani Asintomatici':list(giovani_a.values()),
                 'CI bottom':[CI_giovani_a[i][0] for i in CI_giovani_a],
                 'CI top':[CI_giovani_a[i][1] for i in CI_giovani_a]}))
# Make a data frame
dataframes.append(pd.DataFrame({'x': range(1,len(simulations[0]['simulation'])+1),'Giovani Sintomatici':list(giovani_s.values()),
                 'CI bottom':[CI_giovani_s[i][0] for i in CI_giovani_s],
                 'CI top':[CI_giovani_s[i][1] for i in CI_giovani_s]}))
# Make a data frame
dataframes.append(pd.DataFrame({'x': range(1,len(simulations[0]['simulation'])+1),'Attivi Asintomatici':list(attivi_a.values()),
                 'CI bottom':[CI_attivi_a[i][0] for i in CI_attivi_a],
                 'CI top':[CI_attivi_a[i][1] for i in CI_attivi_a]}))
# Make a data frame
dataframes.append(pd.DataFrame({'x': range(1,len(simulations[0]['simulation'])+1),'Attivi Sintomatici':list(attivi_s.values()),
                 'CI bottom':[CI_attivi_s[i][0] for i in CI_attivi_s],
                 'CI top':[CI_attivi_s[i][1] for i in CI_attivi_s]}))
# Make a data frame
dataframes.append(pd.DataFrame({'x': range(1,len(simulations[0]['simulation'])+1),'Anziani Asintomatici':list(anziani_a.values()),
                 'CI bottom':[CI_anziani_a[i][0] for i in CI_anziani_a],
                 'CI top':[CI_anziani_a[i][1] for i in CI_anziani_a]}))
# Make a data frame
dataframes.append(pd.DataFrame({'x': range(1,len(simulations[0]['simulation'])+1),'Anziani Sintomatici':list(anziani_s.values()),
                 'CI bottom':[CI_anziani_s[i][0] for i in CI_anziani_s],
                 'CI top':[CI_anziani_s[i][1] for i in CI_anziani_s]}))

In [35]:
# style
plt.clf()
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(15,10))
 
# create a color palette
palette = plt.get_cmap('Set1')
 
# multiple line plot
num = 0
for df in dataframes:
    num += 1
    for column in df.drop('x', axis=1):
        if column != 'CI bottom' and column != 'CI top':
            plt.plot(df['x'], df[column], marker='', color=palette(num), linewidth=1, alpha=0.9, label=column)
            plt.plot(df['x'], df['CI bottom'], marker='', color=palette(num), linewidth=0, alpha=0)
            plt.plot(df['x'], df['CI top'], marker='', color=palette(num), linewidth=0, alpha=0)
            plt.fill_between(df['x'], df['CI bottom'], df['CI top'], color=palette(num), alpha=0.3, linewidth=0)
                                                             

#Add legend
plt.legend()

# general title
plt.suptitle("Andamento dell'infezione A/S per fascia di età", fontsize=13, fontweight=0, color='black', style='italic', y=0.92)
    
#Add titles
plt.xlabel("Time")
plt.ylabel("Score")
plt.show()
plt.savefig('export_multi_full/noschools_progression_single.png')

In [36]:
# Initialize the figure
plt.clf()
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(15,10))

# create a color palette
palette = plt.get_cmap('Set1')
 
# multiple line plot

num = 0
for df in dataframes:
    num += 1
    for column in df.drop('x', axis=1):
        if column != 'CI bottom' and column != 'CI top':
            plt.subplot(2,3, num)
            plt.plot(df['x'], df[column], marker='', color=palette(num), linewidth=1, alpha=0.9, label=column)
            plt.plot(df['x'], df['CI bottom'], marker='', color=palette(num), linewidth=0, alpha=0)
            plt.plot(df['x'], df['CI top'], marker='', color=palette(num), linewidth=0, alpha=0)
            plt.fill_between(df['x'], df['CI bottom'], df['CI top'], color=palette(num), alpha=0.3)
            
            # Same limits for everybody!
            plt.xlim(0,61)
            plt.ylim(0,25000)
        
            # Not ticks everywhere
            if num in range(7) :
                plt.tick_params(labelbottom='off')
            if num not in [1,4,7] :
                plt.tick_params(labelleft='off')

            # Add title
            plt.title(column, loc='left', fontsize=12, fontweight=0, color=palette(num))
            
# general title
plt.suptitle("Andamento dell'infezione A/S per fascia di età", fontsize=13, fontweight=0, color='black', style='italic', y=0.98)
 
# Axis title
plt.text(0.5, 0.02, 'Time', ha='center', va='center')
plt.text(0.06, 0.5, 'Note', ha='center', va='center', rotation='vertical')
plt.show()
plt.savefig('export_multi_full/noschools_progression_multi.png')